In [1]:
# Import libraries
import json
import numpy as np
import pandas as pd

In [2]:
# Select technology
TECH = 'C'

In [3]:
# Load settings for technology
with open(f"../data/tech{TECH}/settings.json") as sfile:
    settings = json.load(sfile)

In [4]:
# Load tech B time series data
colnames = ["addr", "time", "r", "g"]
data = pd.read_csv(f"../data/tech{TECH}/relaxdata.tsv.gz", names=colnames, sep='\t')

In [5]:
# Check data
data.head()

,addr,time,r,g
0,10354,0.000000,2.082326e+06,4.802322e-07
1,10354,0.006997,5.372904e+06,1.861191e-07
2,10354,0.016950,5.513018e+06,1.813889e-07
3,10354,0.023997,5.602565e+06,1.784897e-07
4,10354,0.030978,5.537474e+06,1.805878e-07


In [6]:
# Subtract initial time and compute range and gi
data["time"] -= data.groupby("addr")["time"].transform("first") 
data["gi"] = data.groupby("addr")["g"].transform("first")
data["range"] = np.int32(data["gi"] / settings["gmax"] * 32)

In [7]:
# Check data as needed
data.head()
# data.groupby('range')['range'].count() / 5500

,addr,time,r,g,gi,range
0,10354,0.000000,2.082326e+06,4.802322e-07,4.802322e-07,0
1,10354,0.006997,5.372904e+06,1.861191e-07,4.802322e-07,0
2,10354,0.016950,5.513018e+06,1.813889e-07,4.802322e-07,0
3,10354,0.023997,5.602565e+06,1.784897e-07,4.802322e-07,0
4,10354,0.030978,5.537474e+06,1.805878e-07,4.802322e-07,0


In [8]:
# Get times
tdata = []
for time in settings["times"]:
    idx = (data["time"] - time).abs().groupby(data["addr"]).idxmin()
    d = data[data.index.isin(idx)]
    print(len(d))
    d = d[(d["time"] <= time*1.2) & (d["time"] >= time*0.8)]
    print(len(d))
    d["timept"] = time
    tdata.append(d)
tdata = pd.concat(tdata)

16292
16292
16292
16205
16292
16290
16292
16292
16292
7744
16292
16292


In [9]:
# Write to file
fopts = {"sep": '\t', "header": False, "index": False}
tdata.to_csv(f"../data/tech{TECH}/relaxdata.min.tsv.gz", **fopts)